In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Set working directory
os.chdir(
    "Z:\\File folders\\Teaching\\Reproducible Research\\2023\\Repository\\RRcourse2023\\6. Coding and documentation"
)

In [ ]:
# Funtions
def read_employment_data(file_path):
    isco_data = {}
    for i in range(1, 10):
        isco_data[i] = pd.read_excel(file_path, sheet_name=f"ISCO{i}")
    return isco_data


def calculate_country_totals(isco_data):
    country_totals = {}
    for country, data in isco_data.items():
        country_totals[country] = data.sum(axis=1)
    return country_totals


def merge_all_data(isco_data):
    all_data = pd.concat(isco_data.values(), ignore_index=True)
    all_data["ISCO"] = all_data.index // len(isco_data[1]) + 1
    return all_data


def calculate_country_shares(all_data, country_totals):
    country_shares = {}
    for country, total in country_totals.items():
        country_shares[f"share_{country}"] = all_data[country] / total.values
    return country_shares


def calculate_standardized_values(combined, tasks, country_shares):
    standardized_columns = {}
    for country, share in country_shares.items():
        for task in tasks:
            temp_mean = np.average(combined[task], weights=share)
            temp_sd = np.sqrt(
                np.average((combined[task] - temp_mean) ** 2, weights=share)
            )
            col_name = f"std_{country}_{task}"
            combined[col_name] = (combined[task] - temp_mean) / temp_sd
            standardized_columns[col_name] = combined[col_name]
    return standardized_columns


def calculate_task_content_intensity(combined, tasks, country_names):
    for country in country_names:
        combined[f"{country}_NRCA"] = sum(
            combined[f"std_{country}_{task}"] for task in tasks
        )


def calculate_weighted_mean(combined, task_intensity, country_shares):
    weighted_means = {}
    for country, share in country_shares.items():
        weighted_means[f"multip_{country}_{task_intensity}"] = (
            combined[f"std_{country}_{task_intensity}"] * share
        )
    return weighted_means


def plot_task_intensity_over_time(agg_data, country_name):
    plt.plot(agg_data["TIME"], agg_data[f"multip_{country_name}_NRCA"])
    plt.xticks(range(0, len(agg_data), 3), agg_data["TIME"][::3])
    plt.title(
        f"{country_name} Non-routine Cognitive Analytical Task Intensity Over Time"
    )
    plt.xlabel("Time")
    plt.ylabel("Task Intensity")
    plt.show()

In [ ]:
# Read employment data
isco_data = read_employment_data("Data\\Eurostat_employment_isco.xlsx")

# Calculate country totals
country_totals = calculate_country_totals(isco_data)

# Merge all data
all_data = merge_all_data(isco_data)

# Calculate country shares
country_shares = calculate_country_shares(all_data, country_totals)

# Read task data
task_data = pd.read_csv("Data\\onet_tasks.csv")
task_data["isco08_1dig"] = task_data["isco08"].astype(str).str[:1].astype(int)

# Calculate mean task values
aggdata = task_data.groupby(["isco08_1dig"]).mean().drop(columns=["isco08"])

# Merge task data with all data
combined = pd.merge(
    all_data, aggdata, left_on="ISCO", right_on="isco08_1dig", how="left"
)

In [ ]:
# Tasks of interest
tasks_of_interest = ["t_4A2a4", "t_4A2b2", "t_4A4a1"]

# Calculate standardized values
standardized_columns = calculate_standardized_values(
    combined, tasks_of_interest, country_shares
)

# Calculate task content intensity
calculate_task_content_intensity(
    combined, tasks_of_interest, ["Belgium", "Poland", "Spain"]
)

# Calculate weighted means
weighted_means = {}
for country, share in country_shares.items():
    for task_intensity in ["Belgium_NRCA", "Poland_NRCA", "Spain_NRCA"]:
        weighted_means.update(
            calculate_weighted_mean(combined, task_intensity, country_shares)
        )

# Aggregate data over time
agg_data = {}
for country in ["Belgium", "Poland", "Spain"]:
    agg_data[country] = (
        combined.groupby(["TIME"])[f"multip_{country}_NRCA"].sum().reset_index()
    )

# Plot task intensity over time for each country
for country in ["Belgium", "Poland", "Spain"]:
    plot_task_intensity_over_time(agg_data[country], country)